# Capstone: BERT Twitter Market Prediction

### Jordan Bickelhaupt, Evelina Ramoskaite, Dave Sawh, Andrew Schiek
###### jordanrbickelhaupt@gmail.com, evelina.ramoskaite@gmail.com, devindrasawh@gmail.com, schiekandrew@gmail.com


Data: https://www.kaggle.com/datasets/kazanova/sentiment140


### Imports & Dependencies

In [1]:
# !pip install tensorflow
# !pip install tensorflow_hub==0.9.0
# !pip install keras tf-models-official pydot graphviz
# !pip install yfinance
# !pip install bert-tensorflow==1.0.1
# !pip install -q -U "tensorflow-text==2.8.*"

In [2]:
import official.nlp.optimization as op


In [3]:
import os
import sys
print(sys.version)
os.getcwd()

3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bit (AMD64)]


'C:\\Users\\jbickelhaupt\\Downloads\\BERT-main (4)\\BERT-main'

3.8.8 (default, Apr 13 2021, 19:58:26) 
[GCC 7.3.0]

'/home/jordan/rot/ent/tfBERT'

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub

# import official.nlp.bert.bert_models
# import official.nlp.bert.configs
# import official.nlp.bert.run_classifier
# import official.nlp.bert.tokenization as tokenization
import bert.tokenization as tokenization

from official.modeling import tf_utils
from official import nlp
# from official.nlp import bert

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   try:
#     # Currently, memory growth needs to be the same across GPUs
#     for gpu in gpus:
#       tf.config.experimental.set_memory_growth(gpu, True)
#     logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     # Memory growth must be set before GPUs have been initialized
#     print(e)

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.8.3
Eager mode:  True
Hub version:  0.9.0
GPU is available


In [5]:
# tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# if gpus:
#   try:
#     # Currently, memory growth needs to be the same across GPUs
#     for gpu in gpus:
#       tf.config.experimental.set_memory_growth(gpu, True)
#     logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     # Memory growth must be set before GPUs have been initialized
#     print(e)


Num GPUs Available:  1


# Data

##### Sentiment 140 dataset
https://www.tensorflow.org/datasets/catalog/sentiment140

Context \\
\
This is the sentiment140 dataset. It contains 1,600,000 tweets extracted using the twitter api . The tweets have been annotated (0 = negative, 4 = positive) and they can be used to detect sentiment. \\
\
Content \\
\
It contains the following 6 fields:
* target: the polarity of the tweet (0 = negative, 4 = positive)

* ids: The id of the tweet ( 2087)

* date: the date of the tweet (Sat May 16 23:58:44 UTC 2009)

* flag: The query (lyx). If there is no query, then this value is NO_QUERY.

* user: the user that tweeted (robotickilldozr)

* text: the text of the tweet (Lyx is cool)

Acknowledgements \\
The official link regarding the dataset with resources about how it was generated is [here](http://%20http//help.sentiment140.com/for-students/) \\
The official paper detailing the approach is [here](http://bhttp//cs.stanford.edu/people/alecmgo/papers/TwitterDistantSupervision09.pdf)

Citation: Go, A., Bhayani, R. and Huang, L., 2009. Twitter sentiment classification using distant supervision. CS224N Project Report, Stanford, 1(2009), p.12.

In [6]:
import random
df = pd.read_csv('sent140.csv',                                           
                 encoding='ISO-8859-1', 
                 names=[
                        'target',
                        'id',
                        'date',
                        'flag',
                        'user',
                        'text'
                        ])

## Train/test split

We don't really need all 1.6 Million tweets for training so we can take a sample of 5% (to save up time on training) and then split that into 80% for training and 20% for testing.

In [7]:
df['wkd'] = [i[0:3] for i in df['date']]


sample_size = int(len(df)*0.0005)
sampleDf = df.sample(sample_size, random_state=23)
wkd = sampleDf.wkd.values
x = sampleDf.text.values
y = sampleDf.target.values
wkd_train, wkd_test, x_train, x_test, y_train, y_test = train_test_split(wkd, x, y, test_size=0.20, random_state=32)

In [8]:
from keras.utils import np_utils
# If np_utils fails to load, rerun cell

## Label Encoding

We could get away with other approach but you might want to use this for more than binary classification.

In [9]:
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y_test = encoder.transform(y_test)
encoded_Y_train = encoder.transform(y_train)

# convert integers to dummy variables (i.e. one hot encoded)
dummy_y_test = np_utils.to_categorical(encoded_Y_test)
dummy_y_train = np_utils.to_categorical(encoded_Y_train)

We might want to use encoding for later. For that we can save enconding map.

In [10]:
encoder_fname = 'twitter_classes.npy'
my_wd = 'C:\\Users\\jbickelhaupt\\Downloads\\BERT-main (4)\\BERT-main'
np.save(os.path.join(my_wd, encoder_fname) , encoder.classes_)

To load it when you'll use this in production just use the below cell (uncommented ofcourse)

In [11]:
# encoder = LabelEncoder()
# encoder.classes_ = np.load(os.path.join(my_wd, encoder_fname), allow_pickle=True)

## Tokenization

To tokenize our text we will use some functions from official.nlp.bert package and the pretrained BERT model itself. \\
First we get the BERT model. I'll use the multilingual one as it will be best for most of your cases. To check all NLP models provided by TF HUB go [here](https://tfhub.dev/s?module-type=text-embedding,text-classification,text-generation,text-language-model,text-question-answering,text-retrieval-question-answering).

In [12]:
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/2",
                            trainable=True)

In [13]:
import sys
from absl import flags
sys.argv=['preserve_unused_tokens=False']
flags.FLAGS(sys.argv)

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

You can see that in above cell we have loaded some variables using the bert_layer we have downloaded. 

1.   ```vocab_file``` reads the vocab file associated to the downloaded model.
2.   ```do_lower_case``` reads binary variable which if ```True``` means tokenizer will reformat all text to lower case rendering model to be **NOT** case sensitive. Should be ```False``` by default. You can check in a cell below.
3.   ```tokenizer``` builds tokenizer using variables 1 and 2.



Lastly we need to add two additional tokens: Classification and Seperation. We will add these through functions we will use to tokenize our text.

In [14]:
tokenizer.convert_tokens_to_ids(['[CLS]', '[SEP]'])

[101, 102]

Kewl, by this point we have everything for tokenization. This will take a bit as we have quite a bit of data.

## Prep inputs

So since we went extensively through input prep in last video where we finetune non modified BERT model, this time we'll jump straight to input prep through functions. Everything that will be different from the old notebook will have **[NEW]** next to it. Our inputs are:

1.   Tokens
2.   Input mask
3.   Input type
4.   **[NEW]** Weekday

Remember when I said it will make our work easier both now and in the future if we remake everything into funcitons? Now is that future.



First let's set the max sequence lenght as we did in simple NN example.

In [15]:
def encode_names(n, tokenizer):
   tokens = list(tokenizer.tokenize(n))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)

tweets = tf.ragged.constant([encode_names(n, tokenizer) for n in x_train])
cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*tweets.shape[0]
input_word_ids = tf.concat([cls, tweets], axis=-1)

lens = [len(i) for i in input_word_ids]

In [16]:
max_seq_length = max(lens)
print('Max length is:', max_seq_length)

Max length is: 105


Most of the time I suggest adding a bit more to the `max_seq_length` esspecially when ussing just a fraction of data. Let's make it `1.5*max_seq_length`.

In [17]:
max_seq_length = int(1.5*max_seq_length)
print('Max length is:', max_seq_length)

Max length is: 157


**[NEW]** We need to encode our `wkd` classes and update our input functions to include that.

In [18]:
featureEncoder = LabelEncoder()
featureEncoder.fit(wkd)

# encode
encoded_wkd_train = featureEncoder.transform(wkd_train)
encoded_wkd_test = featureEncoder.transform(wkd_test)

# convert integers to dummy variables (i.e. one hot encoded)
dummy_wkd_train = np_utils.to_categorical(encoded_wkd_train)
dummy_wkd_test = np_utils.to_categorical(encoded_wkd_test)

**[NEW]** And once again save this encoder for later as we did with labels.

In [19]:
featureEncoder_fname = 'twitter_wkd.npy'
np.save(os.path.join(my_wd, featureEncoder_fname) , featureEncoder.classes_)

Now we can make our inputs for train and test subsamples. There will be **[NEW]** as a comment to new lines which differ from our last notebook.

In [20]:
def encode_names(n, tokenizer):
   tokens = list(tokenizer.tokenize(n))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)

def bert_encode(string_list,
                tokenizer, 
                new_feature,  # [NEW]
                new_feature_class_count,  # [NEW] 
                max_seq_length):  
  num_examples = len(string_list)
  
  string_tokens = tf.ragged.constant([
      encode_names(n, tokenizer) for n in np.array(string_list)])

  cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*string_tokens.shape[0]
  input_word_ids = tf.concat([cls, string_tokens], axis=-1)

  input_mask = tf.ones_like(input_word_ids).to_tensor(shape=(None, max_seq_length))

  type_cls = tf.zeros_like(cls)
  type_tokens = tf.ones_like(string_tokens)
  input_type_ids = tf.concat(
      [type_cls, type_tokens], axis=-1).to_tensor(shape=(None, max_seq_length))
  feature = tf.ragged.constant(new_feature).to_tensor(shape=(None, new_feature_class_count))  # [NEW]

  inputs = {
      'input_word_ids': input_word_ids.to_tensor(shape=(None, max_seq_length)),
      'input_mask': input_mask,
      'input_type_ids': input_type_ids,
      'additional_feature': feature}  # [NEW]

  return inputs

**[NEW]** And now we preprocess inputs.

In [21]:
feature_class_count = len(df.wkd.unique())
X_train = bert_encode(x_train, tokenizer, dummy_wkd_train, feature_class_count, max_seq_length)
X_test = bert_encode(x_test, tokenizer, dummy_wkd_test, feature_class_count, max_seq_length)

# Bam MODEL part already

## Initial training

We need to set up our model using the inputs we made, BERT model that we downloaded and an output layer based on num of classes we are using. Check for **[NEW]** lines to understand the addition to our inputs.

In [22]:
num_class = len(encoder.classes_)  # Based on available class selection
max_seq_length = max_seq_length  # we calculated this a couple cells ago

input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")
feature_input = tf.keras.layers.Input(shape=(feature_class_count,),  # [NEW]
                                      dtype=tf.float32, 
                                      name="additional_feature")

pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])    

output = tf.keras.layers.concatenate([pooled_output, feature_input], name='pooled_with_additional')  # [NEW]

output = tf.keras.layers.Dropout(rate=0.1)(output)  # [NEW]

# additional dense layer here

output = tf.keras.layers.Dense(num_class, activation='softmax', name='output')(output)

model = tf.keras.Model(
    inputs={
        'input_word_ids': input_word_ids,
        'input_mask': input_mask,
        'input_type_ids': segment_ids,
        'additional_feature': feature_input  # [NEW]
        },
        outputs=output)

Our model.

In [23]:
tf.keras.utils.plot_model(model, show_shapes=True, dpi=48)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


**[NEW]** As you can see from the plot above we have two additional layers:


1.   `additional_feature` - our new input layer
2.   `pooled_with_additional` - concatenated layer from `pooled_output` of BERT and our new `additional_feature`



Set up the training parameters.

In [24]:
epochs = 3
batch_size = 20  # select based on your GPU resources
eval_batch_size = batch_size

train_data_size = len(dummy_y_train)
steps_per_epoch = int(train_data_size / batch_size)
num_train_steps = steps_per_epoch * epochs
warmup_steps = int(epochs * train_data_size * 0.1 / batch_size)

optimizer = op.create_optimizer(
    2e-5, num_train_steps=num_train_steps, num_warmup_steps=warmup_steps)

In [25]:
# !pip install 


Compile the model.

In [26]:
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [27]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 157)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 157)]        0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 157)]        0           []                               
                                                                                                  
 keras_layer (KerasLayer)       [(None, 768),        177853441   ['input_word_ids[0][0]',         
                                 (None, 157, 768)]                'input_mask[0][0]',         

Set up a history to check our model performance afterwards.

In [ ]:
history = model.fit(X_train,
                    dummy_y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_test, dummy_y_test),
                    verbose=1)

Epoch 1/3


Evaluation

In [ ]:
loss, accuracy = model.evaluate(X_train, dummy_y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, dummy_y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Noice plots are noice

In [ ]:
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [ ]:
plot_history(history)

So, looking at the plot above we can see that at the 3rd epoch our `Validation Loss` jumped significantly even though `Validation Accuracy` stayed pretty much the same. If we would train it more on the same data it would significantly overtrain. \\
What we can do instead is make another training dataset, lower our learning rate commonly denoted as `lr`, recompile our model using new `lr` and then train it a bit more. \\
But before doing that we should save our model just in case if we would want to do the additional training in later sessions.

## Model saving for later use

Let's save our model for later use. Unfortunatelly we can't use the optimizer we used for model training to save. I get an error and I can't resolve it. So we will recompile our model before saving.

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# model_fname = 'twitter_BERT_wWKD'  # [NEW]

# model.save(os.path.join(my_wd, model_fname))

### Validate saved model

Sometimes TF likes to corrupt your model when saving. It's always a good idea to check if everything loads correctly after save.

In [ ]:
new_model = tf.keras.models.load_model(os.path.join(my_wd, model_fname))

In [ ]:
loss, accuracy = new_model.evaluate(X_test, dummy_y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

In [ ]:
new_model.summary()

For safety reasons I would check the tokenizer variables as well.

In [ ]:
tokenizerSaved = bert.tokenization.FullTokenizer(
    vocab_file=os.path.join(my_wd, model_fname, 'assets/vocab.txt'),
    do_lower_case=False)

If this loads, you should be fine.

In [ ]:
tokenizedTweet = tokenizerSaved.tokenize(x_train[0])
for i in tokenizedTweet:
  print(i, tokenizerSaved.convert_tokens_to_ids([i]))

As everything seems fine, we can move on to second training itteration and try to make our model a bit better.

## Second training itteration

As mentioned before, looking at the plot above we can see that at the 3rd epoch our `Validation Loss` jumped significantly even though `Validation Accuracy` stayed pretty much the same. If we would train it more on the same data it would significantly overtrain. \\

What we can do instead is make another training dataset, lower our learning rate commonly denoted as `lr`, recompile our model using new `lr` and then train it a bit more.

Are there any other options to approach overtraining? Sure. One of the most obvious ways would be to increase dropout layer number or the dropout precentage. You could try playing around with pooling as well, but in essence dropout would be your go to. 

If you do not know what dropout is then in short it 'turns off' preceeding layer neurons before next layer calculation, thus increasing the need for the model to 'abstract' things it sees.

Anyway, in our case we are doing `lr` change and adding new data for the model. Let's see how that goes.

### Data

First we need to prep our data. It's good that we made our processing easier making some functions. Let's take another 5% sample from our data.

In [ ]:
sample_size = int(len(df)*0.05)
sampleDf = df.sample(sample_size, random_state=42)
wkd = sampleDf.wkd.values
x = sampleDf.text.values
y = sampleDf.target.values
wkd_train2, wkd_test2, x_train2, x_test2, y_train2, y_test2 = train_test_split(wkd, x, y, test_size=0.20, random_state=23) 

In [ ]:
classes = sampleDf.target.unique()
print(classes)
counts = []

for i in classes:
  count = len(sampleDf[sampleDf.target==i])
  counts.append(count)

plt.bar(['negative', 'positive'], counts)
plt.show()

It's not a bad idea to always check if your sample is representitive of the dataset itself. Most of the time it will, but when getting small samples it might be off in small datasets.

#### Label Encoding
Now we need to encode labels again. Good thing we have our label encoder saved.

In [ ]:
encoder_fname = 'twitter_classes.npy'

encoder = LabelEncoder()
encoder.classes_ = np.load(os.path.join(my_wd, encoder_fname), allow_pickle=True)

encoded_Y_test2 = encoder.transform(y_test2)
encoded_Y_train2 = encoder.transform(y_train2)

# convert integers to dummy variables (i.e. one hot encoded)
dummy_y_test2 = np_utils.to_categorical(encoded_Y_test2)
dummy_y_train2 = np_utils.to_categorical(encoded_Y_train2)

#### Input preprocessing
As we did before we need to tokenize our inputs (tweets) as `input_word_ids` and then add `input_mask` and `input_type`, as well as, `wkd` for weekday input. As we saved our model, we can use it to build our tokenizer as it was.

In [ ]:
model_fname = 'twitter_BERT_wWKD'

tokenizerSaved = bert.tokenization.FullTokenizer(
    vocab_file=os.path.join(my_wd, model_fname, 'assets/vocab.txt'),
    do_lower_case=False)

And use our feature label encoder that we saved earlier.

In [ ]:
encoder_fname = 'twitter_wkd.npy'

featureEncoderSaved = LabelEncoder()
featureEncoderSaved.classes_ = np.load(os.path.join(my_wd, encoder_fname), allow_pickle=True)

# encode
encoded_wkd_train = featureEncoderSaved.transform(wkd_train2)
encoded_wkd_test = featureEncoderSaved.transform(wkd_test2)

# convert integers to dummy variables (i.e. one hot encoded)
dummy_wkd_train2 = np_utils.to_categorical(encoded_wkd_train)
dummy_wkd_test2 = np_utils.to_categorical(encoded_wkd_test)

I repaste our encoding functions here so it would be easier to follow.

In [ ]:
def encode_names(n, tokenizer):
   tokens = list(tokenizer.tokenize(n))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)

def bert_encode(string_list,
                tokenizer, 
                new_feature,  # [NEW]
                new_feature_class_count,  # [NEW] 
                max_seq_length):  
  num_examples = len(string_list)
  
  string_tokens = tf.ragged.constant([
      encode_names(n, tokenizer) for n in np.array(string_list)])

  cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*string_tokens.shape[0]
  input_word_ids = tf.concat([cls, string_tokens], axis=-1)

  input_mask = tf.ones_like(input_word_ids).to_tensor(shape=(None, max_seq_length))

  type_cls = tf.zeros_like(cls)
  type_tokens = tf.ones_like(string_tokens)
  input_type_ids = tf.concat(
      [type_cls, type_tokens], axis=-1).to_tensor(shape=(None, max_seq_length))
  feature = tf.ragged.constant(new_feature).to_tensor(shape=(None, new_feature_class_count))  # [NEW]

  inputs = {
      'input_word_ids': input_word_ids.to_tensor(shape=(None, max_seq_length)),
      'input_mask': input_mask,
      'input_type_ids': input_type_ids,
      'additional_feature': feature}  # [NEW]

  return inputs

As you can see in the functions we use max_seq_length to convert our tweets into inputs. We can't really change it from the one used to build our model as it is in the model structure itself. See below.

In [ ]:
print('Max sequence length is:', max_seq_length)

In [ ]:
model.summary()

You can see that all of the BERT `InputLayers` have a shape of (0, 240) which is (0, `max_seq_length`). For that reason we must use the same `max_seq_length` for our model to understand the inputs it gets. 

Also, the `additional_feature` input layer has a shape of (0, `feature_class_count`), where `feature_class_count` is 7 as in 7 days in the week.

Now we will use the above functions to make all 4 of our inputs. Do

In [ ]:
feature_class_count = 7  # days in week
X_train2 = bert_encode(x_train2, tokenizerSaved, dummy_wkd_train2, feature_class_count, max_seq_length)
X_test2 = bert_encode(x_test2, tokenizerSaved, dummy_wkd_test2, feature_class_count, max_seq_length)

So here we change only actual inputs of the model but not the contants set by our model architecture which are `feature_class_count` and `max_seq_length`. You could actually just input integers there instead of variables as we know them based on our model.

### Training

Firstly let's load our saved model.

In [ ]:
model_fname = 'twitter_BERT_wWKD'

new_model = tf.keras.models.load_model(os.path.join(my_wd, model_fname))
new_model.summary()

In [ ]:
tf.keras.utils.plot_model(new_model, show_shapes=True, dpi=48)

In [ ]:
# loss, accuracy = new_model.evaluate(X_test, dummy_y_test, verbose=False)  # OLD
# print("Old testing Accuracy:  {:.4f}".format(accuracy))

# loss, accuracy = new_model.evaluate(X_test2, dummy_y_test2, verbose=False)  # NEW
# print("New testing Accuracy:  {:.4f}".format(accuracy))

### NEW TESTING ACCURACY = 0.84

To check training progress we can use the new and the old testing datasets. Their accuracy should be close. Let's see how it compares.

As you can see there's some difference. In any case, I suggest you use the same testing sample to proceed with you training no matter if the accuracy is the same. This of course applies only to when you do not train on the full dataset.

We need to setup our training parameters again.

In [ ]:
epochs = 3
batch_size = 8  # select based on your GPU resources
eval_batch_size = batch_size

train_data_size = len(dummy_y_train2)
steps_per_epoch = int(train_data_size / batch_size)
num_train_steps = steps_per_epoch * epochs
warmup_steps = int(epochs * train_data_size * 0.1 / batch_size)

Now the optimizer we use has learing rate decay in it. Which means learning rate goes down over time by the set parameters. We can lower the initial learning rate for this training session or we could use other optimizer with a fixed learning rate altogether.

In [ ]:
optimizer = nlp.optimization.create_optimizer(
    2e-6, num_train_steps=num_train_steps, num_warmup_steps=warmup_steps)

Let's compile our model.

In [ ]:
new_model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

And now finally we can train again and see if that helps us achieve a better accuracy.

In [ ]:
# history2 = new_model.fit(X_train2,  # using new training set
#                          dummy_y_train2,  # using new training set
#                          epochs=epochs,
#                          batch_size=batch_size,
#                          validation_data=(X_test, dummy_y_test),  # using old test dataset
#                          verbose=1)

Let's plot everything out.

In [ ]:
# for i in history2.history:
#   for ele in history2.history[i]:
#     history.history[i].append(ele)

In [ ]:
#plot_history(history)

Our lines aren't consistent as we changed the training dataset. However, you can see that both the `val_accuracy` and `val_loss` improved. At his point we could itterate again and improve the model a bit more and more. Just understand that every incrimental improvement will take more data/time as it significantly slows down over time.

# Is BERT worth it?

So is BERT worth. BERT is a huge model compared to our simple NN and Logistic Regression models we used before. Is the increase in accuracy impactful enough compared to the increase in computation needed? Let's comapre accuracy:


1.   Logistic Regression 0.80
2.   Simple NN           0.79
3.   Finetuned BERT      0.84

In this case it totally depends on you. I personaly would say let's use Logistic Regression. Easily deployable, low computational resources and the task isn't that signicificant in terms of a mistake. If your goal is maximum accuracy possible I would train BERT a bit more and try to get to at least 85%, that would be significant enough to use BERT instead a much less computationaly intesnive Logistic Regression.



# Test

We need to check in with our label encoder to get our classes.

In [ ]:
encoder_fname = 'twitter_classes.npy'
my_wd = '/home/jordan/rot/ent/tfBERT'
encoder = LabelEncoder()
encoder.classes_ = np.load(os.path.join(my_wd, encoder_fname), allow_pickle=True)

encoder_fname = 'twitter_wkd.npy'
featureEncoderSaved = LabelEncoder()
featureEncoderSaved.classes_ = np.load(os.path.join(my_wd, encoder_fname), allow_pickle=True)

This is how our classes are encoded for the model.

And additional feature encoder.

In [ ]:
featureEncoderSaved.classes_

Same with our tokenizer.

In [ ]:
model_fname = 'twitter_BERT_wWKD'

tokenizerSaved = bert.tokenization.FullTokenizer(
    vocab_file=os.path.join(my_wd, model_fname, 'assets/vocab.txt'),
    do_lower_case=False)

# Import tweets for each asset

### NLP Benchmark Data

In [ ]:
rated_AAL = pd.read_csv('data/rated_AAL.csv')[['timestamp','tweet_text']]
rated_GOOGL = pd.read_csv('data/rated_GOOGL.csv')[['timestamp','tweet_text']]

### NLP & Financial Application

In [ ]:
aal = pd.read_csv('data/hashtagAAL.csv')[['timestamp','tweet_text']]
btc = pd.read_csv('data/hashtagBTC-USD.csv')[['timestamp','tweet_text']]
gme = pd.read_csv('data/hashtagGME.csv')[['timestamp','tweet_text']]
spy = pd.read_csv('data/hashtagSPY.csv')[['timestamp','tweet_text']]
wmt = pd.read_csv('data/hashtagWMT.csv')[['timestamp','tweet_text']]

In [ ]:
def prep_data(df):
    
    newtweets = []
    
    try:
        df['timestamp'] = pd.to_datetime(df['timestamp'], format='%m/%d/%Y %H:%M')
    except:
        df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S')

    #df['tweet_text'] = df['tweet_text'].apply(ast.literal_eval).str.decode("utf-8")
        
    for j in df['tweet_text']:
        j = j.replace("b'","")
        j = j.replace('b"',"")
        if 'https' in j:
            j = j[:j.find('https')]
        newtweets.append(j)
        
    df['tweet_text'] = newtweets
 
    #df['dow'] = df['timestamp'].dt.day_name()

In [ ]:
prep_data(aal)
prep_data(btc)
prep_data(gme)
prep_data(spy)
prep_data(wmt)
prep_data(rated_AAL)
prep_data(rated_GOOGL)

# Generate Sentiment Scores for Benchmark Data

In [ ]:
def sent_tweets(df):
    
    # returns df of tweets with positivity score
    # {'tweets', 'pred_sent'}
    
    preds_total = []
    
    for i in df.tweet_text:

        wkd = ['Mon']
        tweet = [str(i)]

        dummy_wkd = np_utils.to_categorical(featureEncoderSaved.transform(np.array(wkd)))  # encodes weekday

        inputs = bert_encode(string_list=list(tweet),
                          tokenizer=tokenizerSaved,
                          new_feature=dummy_wkd,
                          new_feature_class_count=7,
                          max_seq_length=157)


        preds_total.append(float(new_model.predict(inputs)[0][1]))
             
    sent_tweet = {
        'tweets': df['tweet_text'],
        'pred_sentiment': preds_total          
        
    }
    sent_tweet = pd.DataFrame(sent_tweet, columns = {'tweets', 'pred_sentiment'})


    return sent_tweet

In [ ]:
AALsent = sent_tweets(rated_AAL)
GOOGLsent = sent_tweets(rated_GOOGL)

In [ ]:
prep_data(btc)
btc_sent = sent_tweets(btc[0:100])

In [ ]:
btc_sent.to_csv('sentimentBTC_bert.csv') 
btc_sent.head()

# Generate Sentiment Scores for Financial Application

In [ ]:
def sent_hour(df):
    
    # returns % positivity per hour
    
    preds_hours = []
    
    grp = df.groupby(by=[df.timestamp.map(lambda x : (x.hour))])

    for i in grp.tweet_text:
       
        text = str(i[1]).split('\n')
        
        tweets = []

        for i in text:
            tweets.append(i[5::])

        preds_hour = []        

        for i in tweets:
        
            wkd = ['Mon']
            tweet = [str(i)]

            dummy_wkd = np_utils.to_categorical(featureEncoderSaved.transform(np.array(wkd)))  # encodes weekday

            inputs = bert_encode(string_list=list(tweet),
                              tokenizer=tokenizerSaved,
                              new_feature=dummy_wkd,
                              new_feature_class_count=7,
                              max_seq_length=157)
            
            preds_hours.append(float(new_model.predict(inputs)[0][1]))
            
   
    
    return preds_hours


In [ ]:
preds_aal = sent_hour(aal)
preds_btc = sent_hour(btc)
preds_gme = sent_hour(gme)
preds_spy = sent_hour(spy)
preds_wmt = sent_hour(wmt)

# Financial data to identify % change / hour

In [ ]:
import yfinance as yf

In [ ]:
def get_price(ticker, start, end):
    
    stock = yf.Ticker(f"{ticker}")

    hist = stock.history(period="1d",interval="1h",start=start,end=end)
    
    hist.to_csv(f'data/{ticker}prices.csv')
    
for i in ['AAL','GME','SPY','WMT']:   
    get_price(i,'2021-05-03','2021-05-04')
    
get_price('BTC-USD','2021-04-28','2021-04-29')    

In [ ]:
prices_AAL = pd.read_csv('data/AALprices.csv')
prices_BTC = pd.read_csv('data/BTC-USDprices.csv')
prices_GME = pd.read_csv('data/GMEprices.csv')
prices_SPY = pd.read_csv('data/SPYprices.csv')
prices_WMT = pd.read_csv('data/WMTprices.csv')

In [ ]:
def datax(delta,pred):
    
# pct change data available from 930 until 330 - first value NULL
# preds has tweets for 24 hour day

    pct = delta[1:7]
    preds = pred[8:14]

    newdf = {
        'delta': delta[1:7],
        'pred_sentiment': pred[8:14]         
        
    }
    newdf = pd.DataFrame(newdf, columns = {'delta', 'pred_sentiment'})

    newdf.to_csv('deltaWMT_bert.csv') 

In [ ]:
d_AAL = pd.read_csv('output/deltaAAL_bert.csv')
d_GME = pd.read_csv('output/deltaGME_bert.csv')
d_SPY = pd.read_csv('output/deltaSPY_bert.csv')
d_WMT = pd.read_csv('output/deltaWMT_bert.csv')

# Model Prediction and Stock Δ

In [ ]:
def acc(df):
    df = df.rename(columns = {"Unnamed: 0" : "index", "pred_sentiment" : "pred", "delta":"real"})
    df = df[['real', 'pred']]
    df.real = df.real*100
    df.pred = df.pred
    df.plot()
    plt.xlabel('Stock Market Open Hours')
    plt.ylabel('Δ')
    
#Gamestop was tanking and had difficult to interpret results, 
#however the hourly trend lines were apparent
#acc(d_GME)
acc(d_AAL) 
acc(d_SPY)
acc(d_WMT)